In [11]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [12]:
# Load data
data = pd.read_excel("tournaments_updated.xlsx")

#clean data
data_copy = data[["Tournament", "Date", "Series", "Round", "Winner", "Loser", "WRank", "LRank", "WPts", "LPts", 
"Wpoints", "Fpoints", "SFpoints", "QFpoints", "sixteenpoints", "thirtytwopoints", "sixtyfourpoints", "onehundredtwentyeightpoints"]]

data_copy

,Tournament,Date,Series,Round,Winner,Loser,WRank,LRank,WPts,LPts,Wpoints,Fpoints,SFpoints,QFpoints,sixteenpoints,thirtytwopoints,sixtyfourpoints,onehundredtwentyeightpoints
0,Adelaide International 1,2023-01-01,ATP250,1st Round,Giron M.,Gasquet R.,61.0,68.0,776.0,740.0,250,165,100,50,25,13,0,0
1,Adelaide International 1,2023-01-01,ATP250,1st Round,Mcdonald M.,Galan D.E.,63.0,67.0,775.0,741.0,250,165,100,50,25,13,0,0
2,Adelaide International 1,2023-01-02,ATP250,1st Round,Kecmanovic M.,O Connell C.,29.0,78.0,1420.0,652.0,250,165,100,50,25,13,0,0
3,Adelaide International 1,2023-01-02,ATP250,1st Round,Nishioka Y.,Rune H.,36.0,11.0,1134.0,2888.0,250,165,100,50,25,13,0,0
4,Adelaide International 1,2023-01-02,ATP250,1st Round,Popyrin A.,Auger-Aliassime F.,120.0,6.0,469.0,4195.0,250,165,100,50,25,13,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,Canadian Open,2023-08-12,Masters 1000,Quarterfinals,Paul T.,Alcaraz C.,14.0,1.0,2345.0,9225.0,1000,650,400,200,100,50,30,10
1992,Canadian Open,2023-08-12,Masters 1000,Quarterfinals,Sinner J.,Monfils G.,8.0,276.0,3815.0,207.0,1000,650,400,200,100,50,30,10
1993,Canadian Open,2023-08-12,Masters 1000,Semifinals,De Minaur A.,Davidovich Fokina A.,18.0,37.0,2085.0,1140.0,1000,650,400,200,100,50,30,10
1994,Canadian Open,2023-08-13,Masters 1000,Semifinals,Sinner J.,Paul T.,8.0,14.0,3815.0,2345.0,1000,650,400,200,100,50,30,10


In [13]:
data_copy = data_copy.drop(columns=['Start Date', 'End Date'], errors='ignore')

data_copy['Date'] = pd.to_datetime(data_copy['Date'])

dates = data_copy.groupby('Tournament')['Date']
dates = dates.agg(['min', 'max'])
dates.columns = ['Start Date', 'End Date']

data_copy = data_copy.merge(dates, on='Tournament', how='right')

In [14]:
m = gp.Model("Maximizing Entered Tournaments")

y = {}
for i in range(48):
    y[i] = m.addVar(vtype=GRB.BINARY, name=f"tourney{i}")

m.setObjective(gp.quicksum(y[i] for i in range(48)), GRB.MAXIMIZE)

m.addConstr(gp.quicksum(y[i] for i in range(48)) >= 15)

for i in range(48):
    for j in range(48):
        if i < j:
            start_i = dates.loc[dates.index[i], 'Start Date']
            end_i = dates.loc[dates.index[i], 'End Date']
            start_j = dates.loc[dates.index[j], 'Start Date']
            end_j = dates.loc[dates.index[j], 'End Date']
            if (start_i <= end_j) and (start_j <= end_i):
                m.addConstr(y[i] + y[j] <= 1)

m.optimize()



Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 42 rows, 48 columns and 130 nonzeros
Model fingerprint: 0xa3445eac
Variable types: 0 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 22.0000000
Presolve removed 42 rows and 48 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 1: 22 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.200000000000e+01, best bound 2.200000000000e+01, gap 0.0000%


In [15]:
data_copy.to_csv('final_data.csv', index=False)

In [16]:
#make a row for each player, probabilities for each round, group by tournament, unique players
pd.melt(data_copy, id_vars=['Tournament', 'Start Date', 'End Date', 'Series', 'Round', 'Wpoints', 'Fpoints', 'SFpoints', 'QFpoints', 'sixteenpoints', 'thirtytwopoints', 'sixtyfourpoints', 'onehundredtwentyeightpoints'], value_vars=['Winner', 'Loser'], var_name='PlayerType', value_name='Player')
#for index, row in data_copy.iterrows():
    #data_copy.groupby('Tournament')['Winner', 'Loser'].nunique()

,Tournament,Start Date,End Date,Series,Round,Wpoints,Fpoints,SFpoints,QFpoints,sixteenpoints,thirtytwopoints,sixtyfourpoints,onehundredtwentyeightpoints,PlayerType,Player
0,ABN AMRO World Tennis Tournament,2023-02-13,2023-02-19,ATP500,1st Round,500,330,1200,100,50,25,0,0,Winner,Gasquet R.
1,ABN AMRO World Tennis Tournament,2023-02-13,2023-02-19,ATP500,1st Round,500,330,1200,100,50,25,0,0,Winner,Dimitrov G.
2,ABN AMRO World Tennis Tournament,2023-02-13,2023-02-19,ATP500,1st Round,500,330,1200,100,50,25,0,0,Winner,Wawrinka S.
3,ABN AMRO World Tennis Tournament,2023-02-13,2023-02-19,ATP500,1st Round,500,330,1200,100,50,25,0,0,Winner,Barrere G.
4,ABN AMRO World Tennis Tournament,2023-02-13,2023-02-19,ATP500,1st Round,500,330,1200,100,50,25,0,0,Winner,Brouwer G.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3987,Wimbledon,2023-07-03,2023-07-16,Grand Slam,Quarterfinals,2000,1300,800,400,200,100,50,10,Loser,Eubanks C.
3988,Wimbledon,2023-07-03,2023-07-16,Grand Slam,Quarterfinals,2000,1300,800,400,200,100,50,10,Loser,Rune H.
3989,Wimbledon,2023-07-03,2023-07-16,Grand Slam,Semifinals,2000,1300,800,400,200,100,50,10,Loser,Sinner J.
3990,Wimbledon,2023-07-03,2023-07-16,Grand Slam,Semifinals,2000,1300,800,400,200,100,50,10,Loser,Medvedev D.


In [ ]:
#make a row for each player, probabilities for each round, group by tournament, unique players
#winners data frame
winners = data_copy[['Tournament', 'Winner', 'WRank']].rename(columns={'Winner': 'Player', 'WRank': 'Rank'})
#losers data frame
losers = data_copy[['Tournament', 'Loser', 'LRank']].rename(columns={'Loser': 'Player', 'LRank': 'Rank'})
#combine together into seperate  players data frame
players = pd.concat([winners, losers], ignore_index=True).drop_duplicates() 
# keep only columns needed in data_copy
col_needed_data = data_copy[['Tournament', 'Wpoints', 'Start Date', 'End Date', 'Series', 'Wpoints', 'Fpoints', 'SFpoints', 'QFpoints', 'sixteenpoints', 'thirtytwopoints', 'sixtyfourpoints', 'onehundredtwentyeightpoints']].drop_duplicates()
#left join with rest of data to get rest of columns
final_data = pd.merge(players, col_needed_data, on=['Tournament'], how='left')
final_data
for index, row in data_copy .iterrows():

,Tournament,Player,Rank,Wpoints,Start Date,End Date,Series,Wpoints,Fpoints,SFpoints,QFpoints,sixteenpoints,thirtytwopoints,sixtyfourpoints,onehundredtwentyeightpoints
0,ABN AMRO World Tennis Tournament,Gasquet R.,45.0,500,2023-02-13,2023-02-19,ATP500,500,330,1200,100,50,25,0,0
1,ABN AMRO World Tennis Tournament,Dimitrov G.,28.0,500,2023-02-13,2023-02-19,ATP500,500,330,1200,100,50,25,0,0
2,ABN AMRO World Tennis Tournament,Wawrinka S.,130.0,500,2023-02-13,2023-02-19,ATP500,500,330,1200,100,50,25,0,0
3,ABN AMRO World Tennis Tournament,Barrere G.,71.0,500,2023-02-13,2023-02-19,ATP500,500,330,1200,100,50,25,0,0
4,ABN AMRO World Tennis Tournament,Brouwer G.,160.0,500,2023-02-13,2023-02-19,ATP500,500,330,1200,100,50,25,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2069,Wimbledon,Medjedovic H.,156.0,2000,2023-07-03,2023-07-16,Grand Slam,2000,1300,800,400,200,100,50,10
2070,Wimbledon,Coppejans K.,188.0,2000,2023-07-03,2023-07-16,Grand Slam,2000,1300,800,400,200,100,50,10
2071,Wimbledon,Humbert U.,39.0,2000,2023-07-03,2023-07-16,Grand Slam,2000,1300,800,400,200,100,50,10
2072,Wimbledon,Cecchinato M.,89.0,2000,2023-07-03,2023-07-16,Grand Slam,2000,1300,800,400,200,100,50,10
